In [ ]:
!pip install spatialdata spatialdata_plot spatialdata_io

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import random
import spatialdata as sd
from pathlib import Path
from spatialdata_io import xenium

from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

/usr/lib/python3.12/importlib/__init__.py:90: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  return _bootstrap._gcd_import(name[level:], package, level)
/usr/local/lib/python3.12/dist-packages/cudf/utils/gpu_utils.py:75: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))


In [4]:
data_path = path_write = "/content/drive/MyDrive/Thesis/Projects/Data/Mouse_Brain_Coronal/data.zarr"
sdata = sd.read_zarr(path_write)
sdata

/tmp/ipython-input-3578476546.py:2: UserWarning: SpatialData is not stored in the most current format. If you want to use Zarr v3, please write the store to a new location using `sdata.write()`.
  sdata = sd.read_zarr(path_write)
/usr/local/lib/python3.12/dist-packages/zarr/core/group.py:3530: ZarrUserWarning: Object at zmetadata is not recognized as a component of a Zarr hierarchy.
  warnings.warn(


SpatialData object, with associated Zarr store: /content/drive/MyDrive/Thesis/Projects/Data/Mouse_Brain_Coronal/data.zarr
├── Images
│     ├── 'he_image': DataTree[cyx] (3, 24689, 17051), (3, 12344, 8525), (3, 6172, 4262), (3, 3086, 2131), (3, 1543, 1065)
│     └── 'morphology_focus': DataTree[cyx] (4, 23912, 34154), (4, 11956, 17077), (4, 5978, 8538), (4, 2989, 4269), (4, 1494, 2134)
├── Labels
│     ├── 'cell_labels': DataTree[yx] (23912, 34154), (11956, 17077), (5978, 8538), (2989, 4269), (1494, 2134)
│     └── 'nucleus_labels': DataTree[yx] (23912, 34154), (11956, 17077), (5978, 8538), (2989, 4269), (1494, 2134)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 13) (3D points)
├── Shapes
│     ├── 'cell_boundaries': GeoDataFrame shape: (63173, 1) (2D shapes)
│     ├── 'cell_circles': GeoDataFrame shape: (63173, 2) (2D shapes)
│     └── 'nucleus_boundaries': GeoDataFrame shape: (63036, 1) (2D shapes)
└── Tables
      └── 'table': AnnData (63173, 5006)
with coordi

In [26]:
import warnings
warnings.filterwarnings('ignore')

img = sdata.images['he_image']

# Transform the cell boundaries to global coordinates
navi = sd.transform(
    sdata.shapes['cell_boundaries'],
    to_coordinate_system="global"
)

for ind in tqdm(sdata.tables['table'].obs.index, dynamic_ncols=True):

    id = sdata.tables['table'].obs['cell_id'][int(ind)]

    cell_center = navi.centroid[id]

    # Get cell center (yx format for plotting)
    x, y = cell_center.x, cell_center.y

    # Define crop size (e.g. 290 microns around the cell)
    half_size = 145  # adjust depending on how much context you want
    xmin, xmax = x - half_size, x + half_size
    ymin, ymax = y - half_size, y + half_size

    # Use bounding_box_query to get the cropped SpatialData
    c_img = sd.bounding_box_query(
        img,
        min_coordinate=[xmin, ymin],
        max_coordinate=[xmax, ymax],
        axes=("x", "y"),
        target_coordinate_system="global"
    )

    if type(c_img) == type(e_img):
      ## Convert to numpy
      he_crop_array = c_img['scale0'].image.data.compute()
      normalized_he = he_crop_array.transpose(1, 2, 0)  # (cyx) → (yx, x, c)

    else:
      print(f'id: {id}', type(c_img), 'center: ', cell_center)
      normalized_he = np.zeros_like(normalized_he)

    plt.imsave(f"/content/drive/MyDrive/Thesis/Projects/Data/Mouse_Coronal_Patches/{id}.png", normalized_he)

  0%|          | 0/63173 [00:00<?, ?it/s]

id: aaalgaap-1 <class 'NoneType'> center:  POINT (32.72185865738274 18914.400031160552)
id: aaaneadm-1 <class 'NoneType'> center:  POINT (887.3836322545278 10222.743054372702)
id: aaanijpc-1 <class 'NoneType'> center:  POINT (154.21016819882612 18911.6088542637)
id: aabgacjd-1 <class 'NoneType'> center:  POINT (21180.29711021834 23570.801756719095)
id: abbldkok-1 <class 'NoneType'> center:  POINT (372.01511220815 4312.33807109133)
id: abbmdjhm-1 <class 'NoneType'> center:  POINT (410.46330769964396 4303.530161870463)
id: abfahlbi-1 <class 'NoneType'> center:  POINT (24215.48672312687 252.3777846895144)
id: ieefbmnc-1 <class 'NoneType'> center:  POINT (6886.025051350714 23683.036800526952)
id: kigidanm-1 <class 'NoneType'> center:  POINT (28.73612811438951 18838.096887231346)
id: kigjaeoi-1 <class 'NoneType'> center:  POINT (70.91684609425477 18890.354022236927)
id: kigjeopp-1 <class 'NoneType'> center:  POINT (47.852631943083985 18871.348435343836)
id: kigkcgip-1 <class 'NoneType'> cen

In [6]:
e_img = c_img

In [18]:
normalized_he.shape

(225, 225, 3)